# This notebook is to preprocess the CPI data (source: IMF [http://data.imf.org/regular.aspx?key=61545861])

In [162]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
from time import time
import re
import datetime as dt

%matplotlib inline

%pylab inline
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

# Constants definition
DATA_PATH = '../data/'
PROJECT_ROOT = '../'
SRC = PROJECT_ROOT + 'src/'

EXCHANGES = 'filt_rates.csv'
COUNTRY_CODES = 'filt_codes.csv'
CPI = 'filt_cpi.csv'

sys.path.append(SRC)

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
codes_df = pd.read_csv(DATA_PATH + COUNTRY_CODES, index_col=0)
rates_df = pd.read_csv(DATA_PATH + EXCHANGES, index_col=0)

In [164]:
codes_df.shape

(59, 2)

In [165]:
codes_df

,name,country
code,,
DZD,Algerian Dinar,Algeria
ARS,Argentine Peso,Argentina
AUD,Australian Dollar,Australia
BHD,Bahraini Dinar,Bahrain
BRL,Brazilian Real,Brazil
GBP,Pound Sterling,United Kingdom
BGN,Bulgarian Lev,Bulgaria
CAD,Canadian Dollar,Canada
CLP,Chilean Peso,Chile


In [166]:
# There is no data for Polynesia
codes_df = codes_df.drop('XPF')

# There is no data for Taiwan
codes_df = codes_df.drop('TWD')

In [167]:
cpi_df = pd.read_csv(DATA_PATH + 'cpi.csv')

In [168]:
cpi_filt_df = cpi_df.dropna(axis=0, subset=['code']).set_index('code').replace('...', np.nan)
cpi_filt_df

,country,2007Q1,2007M01,2007M02,2007M03,2007Q2,2007M04,2007M05,2007M06,2007Q3,...,2016M05,2016M06,2016Q3,2016M07,2016M08,2016M09,2016Q4,2016M10,2016M11,2016M12
code,,,,,,,,,,,,,,,,,,,,,
EUR,Euro Area,93.71,93.33,93.6,94.22,95,94.82,95.04,95.14,95.07,...,107.75,107.93,107.53,107.33,107.43,107.84,108.24,108.11,108.02,108.6
CYP,Cyprus,90.84,90.51,90.45,91.56,92.81,92.57,93.03,92.84,92.9,...,100.37,100.22,100.53,100.42,100.33,100.85,101.05,100.94,100.87,101.33
MTL,Malta,90.84,90.53,90.77,91.22,92.03,91.85,92.09,92.16,92.79,...,108.86,109.04,108.91,108.68,108.88,109.17,109.81,109.28,109.71,110.44
SKK,Slovak Republic,92.49,92.33,92.53,92.6,92.89,92.8,92.8,93.07,93.13,...,108.3,108.43,107.97,108.03,107.9,107.97,108.34,108.23,108.37,108.43
AUD,Australia,90.15,NaN,NaN,NaN,91.25,NaN,NaN,NaN,91.89,...,NaN,NaN,113.84,NaN,NaN,NaN,114.46,NaN,NaN,NaN
CAD,Canada,94.65,93.93,94.62,95.39,96.05,95.82,96.25,96.08,96.05,...,110.59,110.85,110.59,110.68,110.5,110.59,110.5,110.85,110.42,110.25
HKD,"China, P.R.: Hong Kong",91.93,92.18,90.95,92.67,92.42,92.18,92.3,92.67,93.15,...,125.06,125.06,125.79,125.55,125.67,126.04,126.65,126.28,126.65,126.89
CZK,Czech Republic,90.2,89.92,90.2,90.48,91.41,91.09,91.43,91.71,92.14,...,108.1,108.16,108.33,108.49,108.33,108.1,108.77,108.44,108.77,109.11
DKK,Denmark,92.59,91.81,92.79,93.19,93.44,93.36,93.52,93.44,93.08,...,107.67,107.78,107.45,107.67,107.35,107.35,107.49,107.56,107.45,107.45


### Australia, New Zealand and the Kingdom of Bahrain seem to repor the CPI quarterly.

In [169]:
cpi_filt_df.isnull().sum(axis=1)

code
EUR      0
CYP      0
MTL      0
SKK      0
AUD    120
CAD      0
HKD      0
CZK      0
DKK      0
ISK      0
ILS      0
JPY      0
KRW      0
NZD    120
NOK      0
SGD      0
SEK      0
CHF      0
GBP      0
CNY      0
FJD      0
INR      0
IDR      0
MYR      0
PHP      0
LKR      0
THB      0
VND      0
BGN      0
HRK      0
HUF      0
PLN      0
RON      0
TRY      0
RUB      0
DZD      0
BHD      7
EGP      0
KWD      0
MAD      0
PKR      0
SAR      0
TND      0
AED      0
GHS      0
ZAR      0
ARS    131
BRL      0
CLP     32
COP      0
HNL      0
JMD      0
MXN      0
PEN      0
TTD      0
UYU      0
VEF     15
dtype: int64

### Let's remove all the countries that have missing values (the idea is not to have a very big dataset, anyway)

In [170]:
no_missing = cpi_filt_df.isnull().sum(axis=1) == 0
cpi_filt_df = cpi_filt_df[no_missing]
cpi_filt_df.shape

(51, 161)

In [171]:
import re

month_matcher = re.compile('\d{4}M\d{2}')

months = [month_matcher.match(col).group(0) for col in cpi_filt_df.columns if month_matcher.match(col) is not None]
print(len(months))

120


In [172]:
cpi_filt_df = cpi_filt_df.loc[:, months]
cpi_filt_df.shape

(51, 120)

In [173]:
cpi_filt_df.head()

,2007M01,2007M02,2007M03,2007M04,2007M05,2007M06,2007M07,2007M08,2007M09,2007M10,...,2016M03,2016M04,2016M05,2016M06,2016M07,2016M08,2016M09,2016M10,2016M11,2016M12
code,,,,,,,,,,,,,,,,,,,,,
EUR,93.33,93.6,94.22,94.82,95.04,95.14,94.9,94.97,95.32,95.78,...,107.32,107.36,107.75,107.93,107.33,107.43,107.84,108.11,108.02,108.6
CYP,90.51,90.45,91.56,92.57,93.03,92.84,92.15,92.67,93.89,94.76,...,99.58,99.74,100.37,100.22,100.42,100.33,100.85,100.94,100.87,101.33
MTL,90.53,90.77,91.22,91.85,92.09,92.16,92.28,92.53,93.56,94.39,...,108.13,108.74,108.86,109.04,108.68,108.88,109.17,109.28,109.71,110.44
SKK,92.33,92.53,92.6,92.8,92.8,93.07,93,93.07,93.33,93.86,...,108.03,108.37,108.3,108.43,108.03,107.9,107.97,108.23,108.37,108.43
CAD,93.93,94.62,95.39,95.82,96.25,96.08,96.16,95.91,96.08,95.82,...,109.82,110.16,110.59,110.85,110.68,110.5,110.59,110.85,110.42,110.25


In [174]:
cpi_filt_df.isnull().sum().sum()

0

In [175]:
cpi_filt_df.to_csv(DATA_PATH + CPI)

### Let's transform the month format to a datetime

In [176]:
mex = cpi_filt_df.columns[0]
mex

'2007M01'

In [177]:
def transform_month(weird_str):
    year, month = weird_str.split('M')
    return dt.date(int(year), int(month), 15)

In [178]:
transform_month(mex)

datetime.date(2007, 1, 15)

In [179]:
cpi_filt_df.rename(columns={old:transform_month(old) for old in cpi_filt_df.columns}, inplace=True)

In [180]:
cpi_filt_df = cpi_filt_df.transpose()
cpi_filt_df.index.name = 'date'
cpi_filt_df.head()

code,EUR,CYP,MTL,SKK,CAD,HKD,CZK,DKK,ISK,ILS,...,GHS,ZAR,BRL,COP,HNL,JMD,MXN,PEN,TTD,UYU
date,,,,,,,,,,,,,,,,,,,,,
2007-01-15,93.33,90.51,90.53,92.33,93.93,92.18,89.92,91.81,73.48,88.77,...,61.25,77.33,84.36,84.93,78.29,62.86,85.85,88.75,72.74,77.95
2007-02-15,93.6,90.45,90.77,92.53,94.62,90.95,90.2,92.79,73.79,88.5,...,62,77.22,84.73,85.93,78.97,63.05,86.09,88.97,72.97,78.43
2007-03-15,94.22,91.56,91.22,92.6,95.39,92.67,90.48,93.19,73.53,88.68,...,62.79,77.89,85.04,86.97,79.56,63.79,86.27,89.28,73.2,79.13
2007-04-15,94.82,92.57,91.85,92.8,95.82,92.18,91.09,93.36,73.99,89.12,...,63.76,78.9,85.25,87.75,80.01,64.1,86.22,89.44,73.95,80.1
2007-05-15,95.04,93.03,92.09,92.8,96.25,92.3,91.43,93.52,74.62,89.12,...,64.91,79.34,85.49,88.01,80.33,64.91,85.8,89.89,74.4,80.71


### Let's look that there are no Euro area countries left in the list

In [181]:
codes_df.loc[cpi_filt_df.columns]

,name,country
code,,
EUR,Euro,European Union
CYP,Cyprus Pound,Cyprus
MTL,Maltese Lira,Malta
SKK,Slovak Koruna,Slovakia
CAD,Canadian Dollar,Canada
HKD,Hong Kong Dollar,Hong Kong
CZK,Czech Koruna,Czech Republic
DKK,Danish Krone,Denmark
ISK,Iceland Krona,Iceland


It would be reasonable, first, to add the region data, and only look for "Europe"

Let's do it in the next notebook.

In [182]:
cpi_filt_df.to_csv(DATA_PATH + CPI)

### Let's filter the rates and codes to contain only the countries that are left now

In [183]:
rates_df.columns.name = 'code'
rates_df = rates_df[cpi_filt_df.columns]
rates_df.head()

code,EUR,CYP,MTL,SKK,CAD,HKD,CZK,DKK,ISK,ILS,...,GHS,ZAR,BRL,COP,HNL,JMD,MXN,PEN,TTD,UYU
date,,,,,,,,,,,,,,,,,,,,,
2007-01-15,1.29890,2.2461,3.0251,0.037374,0.85039,0.12820,0.046615,0.17427,0.014276,0.23633,...,1.08308,0.139034,0.46759,0.000447,0.052925,0.014866,0.091271,0.31326,0.15799,0.041107
2007-02-15,1.30822,2.2580,3.0478,0.037938,0.85422,0.12801,0.046315,0.17551,0.014855,0.23710,...,1.08057,0.139326,0.47722,0.000449,0.052925,0.014818,0.090953,0.31347,0.15842,0.041219
2007-03-15,1.32449,2.2858,3.0879,0.039208,0.85600,0.12799,0.047220,0.17779,0.014932,0.23806,...,1.07854,0.136017,0.47857,0.000455,0.052925,0.014791,0.089954,0.31394,0.15898,0.041350
2007-04-15,1.35185,2.3315,3.1497,0.040393,0.88164,0.12795,0.048273,0.18138,0.015281,0.24494,...,1.07867,0.140786,0.49236,0.000467,0.052925,0.014749,0.091066,0.31463,0.15895,0.041681
2007-05-15,1.35169,2.3265,3.1488,0.040061,0.91333,0.12790,0.047850,0.18140,0.015870,0.24976,...,1.07732,0.142462,0.50388,0.000499,0.052923,0.014682,0.092396,0.31562,0.15918,0.041819


In [188]:
rates_df.to_csv(DATA_PATH + EXCHANGES)

In [186]:
codes_df = codes_df.loc[cpi_filt_df.columns]
print(codes_df.shape)
codes_df.head()

(51, 2)


,name,country
code,,
EUR,Euro,European Union
CYP,Cyprus Pound,Cyprus
MTL,Maltese Lira,Malta
SKK,Slovak Koruna,Slovakia
CAD,Canadian Dollar,Canada


In [189]:
codes_df.to_csv(DATA_PATH + COUNTRY_CODES)